In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle

In [5]:
#Load Images
def load_images_and_labels(folder, label):
    images = []
    for file in os.listdir(folder):
        image_path = os.path.join(folder, file)
        image = tf.keras.utils.img_to_array(tf.keras.utils.load_img(image_path))
        images.append(image.flatten())
    labels = [label] * len(images)
    return np.array(images), np.array(labels)

In [8]:
#converts array [R][G][B]
def process_rgb_columns(features_df):
    rgb_columns = ['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5', 'Cluster 6']
    processed_columns = []

    for col in rgb_columns:
        rgb_split = features_df[col].str.split(r'[-/]', expand=True)
        rgb_split.columns = [f"{col}_R", f"{col}_G", f"{col}_B"]
        
        rgb_split = rgb_split.astype(int)
        processed_columns.append(rgb_split)
    processed_df = pd.concat(processed_columns, axis=1)
    return processed_df


# Load images + labels
train_jaundiced_images, train_jaundiced_labels = load_images_and_labels("./train/train J", 1)
train_healthy_images, train_healthy_labels = load_images_and_labels("./train/train N", 0)


train_jaundiced_features_df = pd.read_csv("./features/jaundiced_features.csv")
train_healthy_features_df = pd.read_csv("./features/healthy_features.csv")

train_jaundiced_features_df = train_jaundiced_features_df.drop(columns=['Image'])
train_healthy_features_df = train_healthy_features_df.drop(columns=['Image'])

train_jaundiced_processed_features = process_rgb_columns(train_jaundiced_features_df)
train_healthy_processed_features = process_rgb_columns(train_healthy_features_df)

train_jaundiced_features = pd.concat([train_jaundiced_features_df.reset_index(drop=True), train_jaundiced_processed_features], axis=1).values
train_healthy_features = pd.concat([train_healthy_features_df.reset_index(drop=True), train_healthy_processed_features], axis=1).values


train_jaundiced = np.hstack((train_jaundiced_images, train_jaundiced_features))
train_healthy = np.hstack((train_healthy_images, train_healthy_features))

X_train = np.vstack((train_jaundiced, train_healthy))
y_train = np.hstack((train_jaundiced_labels, train_healthy_labels))

X_train, y_train = shuffle(X_train, y_train, random_state=1)

test_jaundiced_images, test_jaundiced_labels = load_images_and_labels("./test/test J", 1)
test_healthy_images, test_healthy_labels = load_images_and_labels("./test/test N", 0)

val_jaundiced_images, val_jaundiced_labels = load_images_and_labels("./validate/validate J", 1)
val_healthy_images, val_healthy_labels = load_images_and_labels("./validate/validate N", 0)

# Combine test and validation (no annotations)
X_test = np.vstack((test_jaundiced_images, test_healthy_images))
y_test = np.hstack((test_jaundiced_labels, test_healthy_labels))

X_val = np.vstack((val_jaundiced_images, val_healthy_images))
y_val = np.hstack((val_jaundiced_labels, val_healthy_labels))


X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce').values
X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce').values
X_val = pd.DataFrame(X_val).apply(pd.to_numeric, errors='coerce').values

X_train = X_train[:, :120000] #need to trim kasi may excess na 24 features diko alam kung saan galing

#prent
print("Jaundiced images shape:", train_jaundiced_images.shape)
print("Jaundiced features shape:", train_jaundiced_features.shape)
print("Healthy images shape:", train_healthy_images.shape)
print("Healthy features shape:", train_healthy_features.shape)
print(X_train.dtype)
print(X_train[:5])

Jaundiced images shape: (399, 120000)
Jaundiced features shape: (399, 24)
Healthy images shape: (497, 120000)
Healthy features shape: (497, 24)
float64
[[177. 114.  79. ... 158. 111.  85.]
 [235. 219. 203. ... 237. 227. 217.]
 [149. 100.  70. ... 109.  72.  45.]
 [ 69.  69.  71. ...   7.   1.   1.]
 [198. 144. 120. ... 209. 164. 135.]]


In [9]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("X_val shape:", X_val.shape)

X_train shape: (896, 120000)
X_test shape: (38, 120000)
X_val shape: (18, 120000)


In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Fit 
X_test = scaler.transform(X_test)       # Transform test data (images only with no annotations)
X_val = scaler.transform(X_val)

In [36]:
model = Sequential([
    Dense(512, activation='relu', input_dim=X_train.shape[1]),  # Combined images + features
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary (J or N || 0 or 1)
])

c:\Users\kenji\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
